# 20151230-predict-income-from-census

Related post:  
https://stharrold.github.io/20151230-predict-income-from-census.html

Purpose: Predict total annual household income.

## Initialization

### Imports

In [1]:
cd ~

/home/samuel_harrold


In [2]:
# Import standard packages.
import os
# Import installed packages.
import matplotlib.pyplot as plt
import pandas as pd
# Import local packages.
# TODO: remove autoreload after testing.
# %load_ext autoreload
# %autoreload 2
# IPython magic.
%matplotlib inline

## Globals

In [5]:
path_static = os.path.join(os.path.expanduser(r'~'), r'stharrold.github.io/content/static')
basename = r'20151230-predict-income-from-census'
path_disk = os.path.abspath(r'/mnt/disk-20151227t211000z/')

## Extract-transform-load

